In [1]:
from dataset import load_dataset
from model import ClimatePINN
from train import plot_comparison
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch
import xarray as xr
from visualisation import visualize_predictions

In [ ]:

# Example usage
checkpoint_path = "checkpoints/run_1.pt"  # Update with your checkpoint path
data_dir = "./data/era_5_data"  # Update with your data directory
visualize_predictions(checkpoint_path, data_dir,72)

Using device: cuda
Loading model with configuration: hidden_dim=64, initial_re=100.0


/home/enzolouv/Documents/ENS PS 2024-2025/cours/projet_IA/PINN_Climate/visualisation.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_